為什麼這個老是報錯? 即便從別的可以正常執行的複製過來還是報錯?

結果是因為 

gemini = ChatGoogleGenerativeAI(model = "gemini-pro", verbose = True, temperature = 0.6, google_api_key = os.environ["GOOGLE_API_KEY"]),

# 最後面多了一個","造成的，切記！！！！！！！

In [48]:
import os
from crewai import Agent, Task, Crew, Process

In [49]:
import textwrap
from IPython.display import HTML, display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [50]:
# 設定4個 LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI
# crewai 使用 Ollama 的方法：windows 只需要打開 ubuntu 視窗即可，不需要執行 litellm；Mac 則甚至不需要啟動任何程式，只要有安裝好 Ollama 即可調用。
# 與 autogen 不同，autogen 要調用 Ollama 需要執行 litellm，多一道步驟，比較麻煩
gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.7),
# Ollama 大模型，可以看要用哪一個
# ollama = Ollama(model="openhermes_assistant")  # 這是用 modelfile 建立的
# ollama 2024/03/12 最強大模型 nous-hermes2
ollama = Ollama(model="wizardlm2")

gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.6,
                             google_api_key = os.environ["GOOGLE_API_KEY"]
                             )


In [51]:
# 設定工具 tools
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search
# 最簡易型的工具定義方式
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

# 從 tools 模組中引入需要的工具
from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.file_tools import FileTools
from tools.search_tools import SearchTools
from tools.sec_tools import SECTools
# 下面添加 tools 時，必須用 類別.方法 加入工具
# BrowserTools.scrape_and_summarize_website
# CalculatorTools.calculate
# FileTools.write_file
# SearchTools.search_internet、SearchTools.search_news
# SECTools.search_10q、SECTools.search_10k
# 下面的是依樣畫葫蘆自行加入2行定義類別，親測可以匯入
from tools.find_papers_arxiv import FindPapersArxiv
from tools.get_top_hackernews_stories import GetTopHackernewsStories
# 然後就可以用下面的函式來使用工具，已測試可以~~~
# FindPapersArxiv.search_arxiv
# GetTopHackernewsStories.get_top_hackernews_stories

In [52]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting 
  actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool],
  debug_mode=True,
  # You can pass an optional llm attribute specifying what mode you wanna use.
  # It can be a local model through Ollama / LM Studio or a remote
  # model like OpenAI, Mistral, Antrophic or others (https://python.langchain.com/docs/integrations/llms/)
  #
  # Examples:
  #
  # from langchain_community.llms import Ollama
  # llm=gemini # was defined above in the file
  #
  # from langchain_openai import ChatOpenAI
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7)
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  debug_mode=True,
  # llm=gemini
  # (optional) llm=ollama_llm
)

In [53]:
# 上面建立 agent 時，可以不指定 llm，這裡才指定
researcher.llm = ollama
writer.llm = ollama

In [54]:
"""
新版本的 crewai 在 Task 的參數多了一個
expected_output="Full blog post of at least 5 paragraphs歐And expressed with traditional chinese.",
"""

'\n新版本的 crewai 在 Task 的參數多了一個\nexpected_output="Full blog post of at least 5 paragraphs歐And expressed with traditional chinese.",\n'

In [55]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your answer MUST be the full blog post of at least 5 paragraphs and finally using taditional chinese writing.""",
  expected_output="Full blog post of at least 5 paragraphs. Andwrite in traditional chinese.",
  agent=writer
)

In [56]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

2024-04-19 22:24:04,107 - 35540 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [57]:
# Get your crew to work!
result = crew.kickoff()

# print("######################")
# print(result)
to_markdown(result)

 [DEBUG]: == Working Agent: Senior Research Analyst
 [INFO]: == Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...
1. Begin by identifying the key areas within AI that are currently experiencing rapid growth and innovation, such as machine learning, natural language processing, robotics, computer vision, etc.
2. Look for recent breakthroughs in these areas, focusing on both technological advancements and their practical applications.
3. Consider the impact of these advancements on various industries, including healthcare, finance, autonomous vehicles, and entertainment.
4. Analyze how these developments are shaping the future of work and society, considering ethical implications and policy considerations.
5. Synthesize this information into a comprehensive analysis report t

KeyboardInterrupt: 